In [69]:
#!pip install -U selenium
#!pip install maya

     |████████████████████████████████| 279 kB 893 kB/s eta 0:00:01
     |████████████████████████████████| 124 kB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 5.0 MB/s eta 0:00:011
     |████████████████████████████████| 489 kB 9.7 MB/s eta 0:00:01
  Created wheel for snaptime: filename=snaptime-0.2.4-py3-none-any.whl size=3594 sha256=1924e8a5c19a87f16dcc09c456fea0d5e9ac5c8dc4135a3b01e61fe325f9b448
  Stored in directory: /Users/kevingao/Library/Caches/pip/wheels/ef/75/ba/deb00489e86fe8ed045887958363a3ddf664227d0bab7f03a2
Successfully built snaptime


In [252]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import pickle
import time
from datetime import datetime
import maya
import re
import os
import pickle
from glob import glob

In [160]:
# get a reference to the download menu. This will run before the page has 
# finished loading, so we stick it in a while loop and just keep looping
# until we're successful.
def find_by_id(element_id):
    while True:
        try:
            target = driver.find_element_by_id(element_id)
        except NoSuchElementException:
            time.sleep(0.2)
            continue
        else:
            break
    return target

def find_by_class(element_class):
    counter_broke = 0
    while True:
        try:
            target = driver.find_element_by_class_name(element_class)
        except NoSuchElementException:
            time.sleep(0.2)
            if (counter_broke > 1):
                break
            counter_broke += 1
            continue
        else:
            break
    return target

def get_link(link):
    driver.implicitly_wait(5)
    try:
        driver.get(link)
    except TimeoutException:
        print("problem loading: " + link)
        driver.refresh()
            

In [146]:
with open("dep_list.txt", "rb") as fp:   # Unpickling
    dep_list = pickle.load(fp)
print(dep_list[0:10])

['matplotlib', 'numpy', 'pandas', 'scipy', 'ipywidgets', 'seaborn', 'voila', 'scikit-learn', 'python-dateutil', 'requests']


In [155]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless') -- using headless mode currently leads to NoSuchElementException 
driver = webdriver.Chrome('/Users/kevingao/Documents/beaut/chromedriver', options=options)

In [163]:
def split_repo_stat(s):
    return int(s.split(" ")[-1].replace(",", ""))

def page_scrape(pack_df, name):
    
    
    pack_row = {}
   
    pack_row["library"] = name

    
    # LIBRARIES.IO
    
    # Stats
    link = "https://libraries.io/pypi/" + name
    try:
        get_link(link)
        time.sleep(np.random.rand() * 3 + 2)
        #driver.get(link)  

        page_region = find_by_class("container")
        content_region = page_region.find_elements_by_xpath("//*[@class='col-md-4 sidebar']")

        stats_region = content_region[0]
        sourcerank = stats_region.find_element_by_tag_name("h2")
        pack_row["SourceRank"] = sourcerank.text.split('\n')[1]

        stats = stats_region.find_elements_by_tag_name("dl")[1]
        split_stats = stats.text.split('\n')
        for i in np.arange(len(split_stats) // 2):
            pack_row[stats.text.split('\n')[i * 2]] = stats.text.split('\n')[i * 2 + 1]

        # Versions
        get_link(link + "/versions")
        time.sleep(np.random.rand() * 3 + 2)
        #driver.get(link + "/versions")
        page_region = find_by_class("col-sm-8")
        tab_ver = page_region.find_element_by_tag_name("tbody")
        ind_ver = tab_ver.find_elements_by_tag_name("tr")

        ver_num, ver_date = [], []
        for i in ind_ver:
            front = i.text.split(" Br")[0]
            split_v_d = front.split(" ", 1)
            ver_num.append(split_v_d[0])
            ver_date.append(maya.parse(split_v_d[1]).datetime())

        pack_row["Version Number"] = ver_num
        pack_row["Version Time"] = ver_date
    except:
        pass
    
    
    # PYPI
    link = "https://pypi.org/project/" + name  
    
    try:
        get_link(link)
        time.sleep(np.random.rand() * 3 + 2)
        #driver.get(link)
        page_region = find_by_class("github-repo-info")
        boxes = page_region.find_elements_by_tag_name("li")

        pack_row["stars"] = split_repo_stat(boxes[0].text)
        pack_row["forks"] = split_repo_stat(boxes[1].text)
        pack_row["Open Issues/PRs"] = split_repo_stat(boxes[2].text)
    except:
        pass

    
    
    # Appending
    pack_df = pack_df.append(pack_row, ignore_index=True)
    
    print(name)
    return pack_df
    
    
'''
package_df = pd.DataFrame()

name = "scipy"
page_scrape(package_df, name)
'''

'\npackage_df = pd.DataFrame()\n\nname = "scipy"\npage_scrape(package_df, name)\n'

In [253]:
start = 0
end = 10


package_df = pd.DataFrame(columns=["library"])

for i in np.arange(start, end):
    package_df = page_scrape(package_df, dep_list[i])

matplotlib
numpy
pandas
scipy
ipywidgets
seaborn
voila
scikit-learn
python-dateutil
requests


In [254]:
df = package_df.copy(deep=True)

In [255]:
def col2int(s):
    multiplier = 1
    if (type(s) != str):
        return
    s = s.replace(",", "")
    if (s[-1] == "K"):
        multiplier = 1000
        s = s[:-1]
    if (s[-1] == "B"):
        if (s[-2] == "MB"):
            multiplier = 1
        if (s[-2] == "KB"):
            multiplier = (10 ** -3)
        if (s[-2] == " B"):
            multiplier = (10 ** -6)
        if (s[-2] == "GB"):
            multiplier = (10 ** 3)
        s = s.split(" ")[0]
        
    return float(s) * multiplier

In [256]:
package_col_need = ['Contributors', 'Dependencies', 'Dependent packages',
       'Dependent repositories', 'Forks', 'Repository size', 'SourceRank', 'Stars',
       'Total releases', 'Watchers']

for col in package_col_need:
    package_df[col] = package_df[col].apply(lambda x: col2int(x))

In [257]:
def two_col_combine_not_null(x):
    if x.first_valid_index() is None:
        return None
    else:
        return x[x.first_valid_index()]

In [258]:
package_df["Stars"] = package_df[["stars", "Stars"]].apply(two_col_combine_not_null, axis=1)
package_df["Forks"] = package_df[["forks", "Forks"]].apply(two_col_combine_not_null, axis=1)

In [259]:
package_df = package_df.drop(["stars", "forks"], axis=1)

In [260]:
package_df

,library,Contributors,Dependencies,Dependent packages,Dependent repositories,First release,Forks,Latest release,Open Issues/PRs,Repository size,SourceRank,Stars,Total releases,Version Number,Version Time,Watchers
0,matplotlib,763.0,0.0,10100.0,39600.0,"Jan 9, 2006",5858.0,about a month ago,1657.0,377.00,28.0,13776.0,68.0,"[3.4.2, 3.4.1, 3.4.0, 3.4.0rc3, 3.4.0rc2, 3.4....","[2021-05-08 05:47:00+00:00, 2021-03-31 08:18:0...",569.0
1,numpy,736.0,0.0,25100.0,84900.0,"Dec 2, 2006",5609.0,9 days ago,2312.0,89.10,29.0,17406.0,100.0,"[1.21.0rc2, 1.21.0rc1, 1.20.3, 1.20.2, 1.20.1,...","[2021-06-08 16:38:00+00:00, 2021-05-24 15:55:0...",558.0
2,pandas,970.0,6.0,14400.0,38400.0,"Dec 25, 2009",12542.0,4 days ago,3666.0,219.00,28.0,30067.0,92.0,"[1.3.0rc1, 1.2.4, 1.2.3, 1.2.2, 1.2.1, 1.2.0, ...","[2021-06-13 19:56:00+00:00, 2021-04-12 18:43:0...",1113.0
3,scipy,680.0,1.0,10400.0,43400.0,"Jul 27, 2010",3752.0,3 days ago,1657.0,128.00,26.0,8302.0,65.0,"[1.7.0rc2, 1.7.0rc1, 1.6.3, 1.6.2, 1.6.1, 1.6....","[2021-06-14 15:26:00+00:00, 2021-06-06 17:23:0...",329.0
4,ipywidgets,NaN,8.0,734.0,7470.0,"Aug 3, 2015",NaN,"Feb 22, 2021",NaN,NaN,17.0,NaN,123.0,"[8.0.0a4, 8.0.0a3, 8.0.0a2, 8.0.0a1, 7.6.3, 7....","[2021-02-22 20:59:00+00:00, 2021-02-10 23:51:0...",NaN
5,seaborn,NaN,4.0,1430.0,6390.0,"Oct 28, 2013",1427.0,"Dec 20, 2020",91.0,NaN,16.0,8505.0,23.0,"[0.11.1, 0.11.0, 0.11.0rc0, 0.10.1, 0.10.0, 0....","[2020-12-20 20:53:00+00:00, 2020-09-08 14:53:0...",NaN
6,voila,23.0,9.0,30.0,13.0,"Aug 8, 2018",330.0,2 months ago,238.0,15.20,15.0,3328.0,58.0,"[0.2.10, 0.2.9, 0.2.8, 0.2.7, 0.2.6, 0.2.5, 0....","[2021-04-28 14:59:00+00:00, 2021-04-13 18:31:0...",79.0
7,scikit-learn,764.0,19.0,4320.0,18900.0,"Sep 22, 2011",21551.0,2 months ago,2361.0,118.00,27.0,46138.0,54.0,"[0.24.2, 0.24.1, 0.24.0, 0.24.0rc1, 0.24.dev0,...","[2021-04-28 11:18:00+00:00, 2021-01-19 18:06:0...",2256.0
8,python-dateutil,103.0,1.0,5620.0,58300.0,"Aug 6, 2008",354.0,"Nov 3, 2019",NaN,5.42,26.0,1500.0,23.0,"[2.8.1, 2.8.0, 2.7.5, 2.7.4, 2.7.3, 2.7.2, 2.7...","[2019-11-03 05:42:00+00:00, 2019-02-05 14:12:0...",42.0
9,requests,563.0,8.0,30100.0,152000.0,"Feb 14, 2011",8320.0,"Dec 16, 2020",327.0,12.00,31.0,45464.0,139.0,"[2.25.1, 2.25.0, 2.24.0, 2.23.0, 2.22.0, 2.21....","[2020-12-16 19:38:00+00:00, 2020-11-11 20:05:0...",1393.0


In [261]:
package_df.to_csv("scraper_" + str(start) + "_" + str(end - 1) + ".csv")

---------------------

In [262]:
stopper

Combining csvs from above.

In [263]:
files = glob("scraper_*")

In [264]:
data = pd.DataFrame()

for file in files:
    temp_df = pd.read_csv(file, index_col=0)
    data = data.append(temp_df)

In [265]:
data

,library,Contributors,Dependencies,Dependent packages,Dependent repositories,First release,Forks,Latest release,Open Issues/PRs,Repository size,SourceRank,Stars,Total releases,Version Number,Version Time,Watchers
0,matplotlib,763.0,0.0,10100.0,39600.0,"Jan 9, 2006",5858.0,about a month ago,1657.0,377.00,28.0,13776.0,68.0,"['3.4.2', '3.4.1', '3.4.0', '3.4.0rc3', '3.4.0...","[datetime.datetime(2021, 5, 8, 5, 47, tzinfo=<...",569.0
1,numpy,736.0,0.0,25100.0,84900.0,"Dec 2, 2006",5609.0,9 days ago,2312.0,89.10,29.0,17406.0,100.0,"['1.21.0rc2', '1.21.0rc1', '1.20.3', '1.20.2',...","[datetime.datetime(2021, 6, 8, 16, 38, tzinfo=...",558.0
2,pandas,970.0,6.0,14400.0,38400.0,"Dec 25, 2009",12542.0,4 days ago,3666.0,219.00,28.0,30067.0,92.0,"['1.3.0rc1', '1.2.4', '1.2.3', '1.2.2', '1.2.1...","[datetime.datetime(2021, 6, 13, 19, 56, tzinfo...",1113.0
3,scipy,680.0,1.0,10400.0,43400.0,"Jul 27, 2010",3752.0,3 days ago,1657.0,128.00,26.0,8302.0,65.0,"['1.7.0rc2', '1.7.0rc1', '1.6.3', '1.6.2', '1....","[datetime.datetime(2021, 6, 14, 15, 26, tzinfo...",329.0
4,ipywidgets,NaN,8.0,734.0,7470.0,"Aug 3, 2015",NaN,"Feb 22, 2021",NaN,NaN,17.0,NaN,123.0,"['8.0.0a4', '8.0.0a3', '8.0.0a2', '8.0.0a1', '...","[datetime.datetime(2021, 2, 22, 20, 59, tzinfo...",NaN
5,seaborn,NaN,4.0,1430.0,6390.0,"Oct 28, 2013",1427.0,"Dec 20, 2020",91.0,NaN,16.0,8505.0,23.0,"['0.11.1', '0.11.0', '0.11.0rc0', '0.10.1', '0...","[datetime.datetime(2020, 12, 20, 20, 53, tzinf...",NaN
6,voila,23.0,9.0,30.0,13.0,"Aug 8, 2018",330.0,2 months ago,238.0,15.20,15.0,3328.0,58.0,"['0.2.10', '0.2.9', '0.2.8', '0.2.7', '0.2.6',...","[datetime.datetime(2021, 4, 28, 14, 59, tzinfo...",79.0
7,scikit-learn,764.0,19.0,4320.0,18900.0,"Sep 22, 2011",21551.0,2 months ago,2361.0,118.00,27.0,46138.0,54.0,"['0.24.2', '0.24.1', '0.24.0', '0.24.0rc1', '0...","[datetime.datetime(2021, 4, 28, 11, 18, tzinfo...",2256.0
8,python-dateutil,103.0,1.0,5620.0,58300.0,"Aug 6, 2008",354.0,"Nov 3, 2019",NaN,5.42,26.0,1500.0,23.0,"['2.8.1', '2.8.0', '2.7.5', '2.7.4', '2.7.3', ...","[datetime.datetime(2019, 11, 3, 5, 42, tzinfo=...",42.0
9,requests,563.0,8.0,30100.0,152000.0,"Feb 14, 2011",8320.0,"Dec 16, 2020",327.0,12.00,31.0,45464.0,139.0,"['2.25.1', '2.25.0', '2.24.0', '2.23.0', '2.22...","[datetime.datetime(2020, 12, 16, 19, 38, tzinf...",1393.0


In [ ]:
data.to_csv("scraper_final.csv")